<a href="https://colab.research.google.com/github/Anjalee13625/pdf-accessible-ai-system/blob/main/pdf_field_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y tesseract-ocr poppler-utils
!pip install -q transformers torch torchvision pytesseract pdf2image pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 63 not upgraded.


In [ ]:
import torch
import pytesseract
import json
from pdf2image import convert_from_path
from PIL import Image
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving dummy pdf.pdf to dummy pdf (2).pdf


In [ ]:
from pdf2image import convert_from_path
import os

# Get uploaded PDF name
pdf_file = list(uploaded.keys())[0]
print("Using PDF:", pdf_file)

# Convert PDF to images
pages = convert_from_path(pdf_file, dpi=200)

print(f"Total pages converted: {len(pages)}")

# Save first page for inspection
pages[0].save("page_1.png", "PNG")

print("Saved page_1.png")


Using PDF: dummy pdf (2).pdf
Total pages converted: 1
Saved page_1.png


In [ ]:
import pytesseract
from PIL import Image
import pandas as pd

# Load the image
page_image = Image.open("page_1.png")

# OCR with bounding boxes
ocr_data = pytesseract.image_to_data(
    page_image,
    output_type=pytesseract.Output.DATAFRAME
)

# Drop empty text rows
ocr_data = ocr_data.dropna().reset_index(drop=True)

print("Total OCR tokens:", len(ocr_data))
ocr_data.head()
ocr_df = pytesseract.image_to_data(
    page_image,
    output_type=pytesseract.Output.DATAFRAME
)

ocr_df = ocr_df.dropna().reset_index(drop=True)
ocr_df.head()



Total OCR tokens: 23


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,5,1,1,1,1,1,200,209,25,22,96,Al
1,5,1,1,1,1,2,235,209,155,22,96,Accessible
2,5,1,1,1,1,3,401,209,56,22,96,PDF
3,5,1,1,1,1,4,467,209,70,22,96,Form
4,5,1,2,1,1,1,203,417,91,22,96,Name:


In [ ]:
# Extract words and bounding boxes
words = []
boxes = []

img_width, img_height = page_image.size

for _, row in ocr_data.iterrows():
    text = str(row["text"]).strip()
    if text == "":
        continue

    words.append(text)

    x0 = row["left"]
    y0 = row["top"]
    x1 = x0 + row["width"]
    y1 = y0 + row["height"]

    # Normalize to 0–1000 (LayoutLM requirement)
    boxes.append([
        int(1000 * x0 / img_width),
        int(1000 * y0 / img_height),
        int(1000 * x1 / img_width),
        int(1000 * y1 / img_height),
    ])

print("Total tokens:", len(words))
print("Total boxes:", len(boxes))


Total tokens: 22
Total boxes: 22


In [ ]:
# Dummy labels (single class for now)
labels = [0] * len(words)

label_list = ["TEXT"]
num_labels = len(label_list)

print("Labels created:", len(labels))


Labels created: 22


In [ ]:
from transformers import LayoutLMv3Processor

processor = LayoutLMv3Processor.from_pretrained(
    "microsoft/layoutlmv3-base",
    apply_ocr=False   # 🔥 THIS FIXES YOUR ERROR
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The image processor of type `LayoutLMv3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


In [ ]:
encoding = processor(
    images=page_image,
    text=words,
    boxes=boxes,
    word_labels=labels,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
)

print("Encoding keys:", encoding.keys())


Encoding keys: KeysView({'input_ids': tensor([[    0,   726,  8076,  4748, 22745,  8575, 10704,    35, 25262,    35,
         36418, 45831, 36418, 23172, 36418,  1944, 25855,    35, 48081, 31886,
         48081, 14969, 13807, 48081,  6494,  2717, 40987,    35, 30072,    35,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1

In [ ]:
import torch
from transformers import LayoutLMv3ForTokenClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    num_labels=num_labels
).to(device)


Using device: cpu


Loading weights:   0%|          | 0/212 [00:00<?, ?it/s]

LayoutLMv3ForTokenClassification LOAD REPORT from: microsoft/layoutlmv3-base
Key                                | Status     | 
-----------------------------------+------------+-
layoutlmv3.embeddings.position_ids | UNEXPECTED | 
classifier.weight                  | MISSING    | 
classifier.bias                    | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
model.eval()

with torch.no_grad():
    outputs = model(
        input_ids=encoding["input_ids"].to(device),
        attention_mask=encoding["attention_mask"].to(device),
        bbox=encoding["bbox"].to(device),
        pixel_values=encoding["pixel_values"].to(device)
    )

logits = outputs.logits
print("Logits shape:", logits.shape)


Logits shape: torch.Size([1, 512, 1])


In [ ]:
model_name = "microsoft/layoutlmv3-base"


In [ ]:
import torch
from transformers import (
    LayoutLMv3Processor,
    LayoutLMv3ForTokenClassification
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
processor = LayoutLMv3Processor.from_pretrained(
    model_name,
    apply_ocr=False
)


In [ ]:
label_list = ["O", "KEY", "VALUE", "HEADER"]
num_labels = len(label_list)


In [ ]:
model = LayoutLMv3ForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels
).to(device)

model.eval()


Loading weights:   0%|          | 0/212 [00:00<?, ?it/s]

LayoutLMv3ForTokenClassification LOAD REPORT from: microsoft/layoutlmv3-base
Key                                | Status     | 
-----------------------------------+------------+-
layoutlmv3.embeddings.position_ids | UNEXPECTED | 
classifier.weight                  | MISSING    | 
classifier.bias                    | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


LayoutLMv3ForTokenClassification(
  (layoutlmv3): LayoutLMv3Model(
    (embeddings): LayoutLMv3TextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
    )
    (patch_embed): LayoutLMv3PatchEmbeddings(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (encoder): LayoutLMv3Encoder

In [ ]:
tokens = ocr_df["text"].tolist()

raw_boxes = ocr_df[["left", "top", "width", "height"]].values.tolist()

print("Total tokens:", len(tokens))


Total tokens: 23


In [ ]:
img_width, img_height = page_image.size

normalized_boxes = []

for box in raw_boxes:
    x, y, w, h = box
    normalized_boxes.append([
        int(1000 * x / img_width),
        int(1000 * y / img_height),
        int(1000 * (x + w) / img_width),
        int(1000 * (y + h) / img_height),
    ])


In [ ]:
print(tokens[:10])
print(normalized_boxes[:3])


['Al', 'Accessible', 'PDF', 'Form', 'Name:', 'Gender:', '()Male', '()', 'Female', '()']
[[120, 89, 136, 98], [142, 89, 235, 98], [242, 89, 276, 98]]


In [ ]:
inputs = processor(
    images=page_image,
    text=tokens,
    boxes=normalized_boxes,
    return_tensors="pt",
    truncation=True,
    padding="max_length"
)

inputs = inputs.to(device)


In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

outputs


TokenClassifierOutput(loss=None, logits=tensor([[[-0.4242, -0.5918, -0.2811,  0.4384],
         [-0.6413, -0.6506, -0.1341, -0.2356],
         [-0.5147, -0.6279, -0.2486, -0.1209],
         ...,
         [-0.4320, -0.6055, -0.2740,  0.4459],
         [-0.4320, -0.6055, -0.2740,  0.4459],
         [-0.4320, -0.6055, -0.2740,  0.4459]]]), hidden_states=None, attentions=None)

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)


In [ ]:
logits = outputs.logits
print("Logits shape:", logits.shape)


Logits shape: torch.Size([1, 512, 4])


In [ ]:
import torch.nn.functional as F

probs = F.softmax(logits, dim=-1)
print("Probabilities shape:", probs.shape)


Probabilities shape: torch.Size([1, 512, 4])


In [ ]:
pred_ids = torch.argmax(probs, dim=-1)

pred_labels = [
    label_list[i] for i in pred_ids[0].cpu().numpy()
]

print("Predicted labels:")
print(pred_labels)


Predicted labels:
['HEADER', 'VALUE', 'HEADER', 'HEADER', 'VALUE', 'HEADER', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'O', 'HEADER', 'HEADER', 'VALUE', 'VALUE', 'VALUE', 'HEADER', 'VALUE', 'HEADER', 'O', 'VALUE', 'VALUE', 'O', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADER', 'HEADE

In [ ]:
hidden_states = outputs.hidden_states if hasattr(outputs, "hidden_states") else None
print("Hidden states available:", hidden_states is not None)


Hidden states available: False


In [ ]:
print(inputs.keys())


KeysView({'input_ids': tensor([[    0,   726,  8076,  4748, 22745,  8575, 10704,    35, 25262,    35,
         36418, 45831, 36418, 23172, 36418,  1944, 25855,    35, 48081, 31886,
         48081, 14969, 13807, 48081,  6494,  2717, 40987,    35,  1437, 30072,
            35,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,

In [ ]:
# Make sure forward pass exists
with torch.no_grad():
    outputs = model(**inputs)

# Extract logits
logits = outputs.logits

# Convert logits → label IDs
predicted_label_ids = torch.argmax(logits, dim=-1)

print("predicted_label_ids shape:", predicted_label_ids.shape)


predicted_label_ids shape: torch.Size([1, 512])


In [ ]:
label_list = [
    "O",        # Outside / irrelevant
    "KEY",      # Field name (e.g. Name, Date)
    "VALUE",    # Field value
    "HEADER"    # Section header
]


In [ ]:
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

print(id2label)



{0: 'O', 1: 'KEY', 2: 'VALUE', 3: 'HEADER'}


In [ ]:
predicted_labels = [
    id2label[i.item()]
    for i in predicted_label_ids[0]
]

print(predicted_labels[:30])


['HEADER', 'VALUE', 'HEADER', 'HEADER', 'VALUE', 'HEADER', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'VALUE', 'O', 'HEADER', 'HEADER', 'VALUE', 'VALUE', 'VALUE', 'HEADER', 'VALUE', 'HEADER', 'O', 'VALUE', 'VALUE', 'O', 'VALUE', 'VALUE', 'VALUE', 'VALUE']


In [ ]:
label_list = ["O", "KEY", "VALUE", "OTHER"]

id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in enumerate(label_list)}


In [ ]:
import torch

model.eval()

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
predicted_label_ids = torch.argmax(logits, dim=-1)


In [ ]:
predicted_labels = [
    id2label[label_id.item()]
    for label_id in predicted_label_ids[0]
]


In [ ]:
list(zip(tokens[:20], predicted_labels[:20]))


[('Al', 'KEY'),
 ('Accessible', 'KEY'),
 ('PDF', 'KEY'),
 ('Form', 'KEY'),
 ('Name:', 'KEY'),
 ('Gender:', 'KEY'),
 ('()Male', 'KEY'),
 ('()', 'KEY'),
 ('Female', 'KEY'),
 ('()', 'KEY'),
 ('Other', 'KEY'),
 ('Skills:', 'OTHER'),
 ('[]', 'OTHER'),
 ('Python', 'OTHER'),
 ('[]', 'OTHER'),
 ('Machine', 'KEY'),
 ('Learning', 'KEY'),
 ('[]', 'KEY'),
 ('Web', 'KEY'),
 ('Development', 'KEY')]

In [ ]:
key_value_pairs = {}

current_key = None
current_value_tokens = []

for token, label in zip(tokens, predicted_labels):
    if label == "KEY":
        if current_key and current_value_tokens:
            key_value_pairs[current_key] = " ".join(current_value_tokens)

        current_key = token
        current_value_tokens = []

    elif label == "VALUE" and current_key is not None:
        current_value_tokens.append(token)

# save last pair
if current_key and current_value_tokens:
    key_value_pairs[current_key] = " ".join(current_value_tokens)


In [ ]:
key_value_pairs


{}

In [ ]:
list(zip(tokens[:30], normalized_boxes[:30]))


[('Al', [120, 89, 136, 98]),
 ('Accessible', [142, 89, 235, 98]),
 ('PDF', [242, 89, 276, 98]),
 ('Form', [282, 89, 324, 98]),
 ('Name:', [122, 178, 177, 187]),
 ('Gender:', [122, 237, 189, 247]),
 ('()Male', [122, 267, 181, 279]),
 ('()', [200, 267, 214, 279]),
 ('Female', [217, 267, 282, 279]),
 ('()', [286, 265, 314, 280]),
 ('Other', [321, 267, 370, 279]),
 ('Skills:', [121, 326, 171, 336]),
 ('[]', [122, 356, 135, 368]),
 ('Python', [142, 356, 200, 368]),
 ('[]', [122, 386, 135, 398]),
 ('Machine', [142, 386, 214, 395]),
 ('Learning', [221, 386, 294, 398]),
 ('[]', [122, 415, 135, 427]),
 ('Web', [141, 415, 179, 424]),
 ('Development', [186, 415, 301, 427]),
 ('Feedback:', [122, 474, 210, 484]),
 (' ', [121, 513, 463, 516]),
 ('Signature:', [121, 563, 207, 575])]

In [ ]:
form_fields = []

for token, box in zip(tokens, normalized_boxes):
    if token.endswith(":"):
        form_fields.append({
            "label": token[:-1],
            "type": "text"
        })

form_fields


[{'label': 'Name', 'type': 'text'},
 {'label': 'Gender', 'type': 'text'},
 {'label': 'Skills', 'type': 'text'},
 {'label': 'Feedback', 'type': 'text'},
 {'label': 'Signature', 'type': 'text'}]

In [ ]:
gender_options = []

for token, box in zip(tokens, normalized_boxes):
    if token in ["Male", "Female", "Other"]:
        gender_options.append(token)

gender_options


['Female', 'Other']

In [ ]:
skills = []

for i, (token, _) in enumerate(zip(tokens, normalized_boxes)):
    if token == "[]":
        skills.append(tokens[i + 1])

skills


['Python', 'Machine', 'Web']

In [ ]:
FIELD_SCHEMA = {
    "name": {
        "label": "Name:",
        "type": "text"
    },
    "gender": {
        "label": "Gender:",
        "type": "radio",
        "options": ["Male", "Female", "Other"]
    },
    "skills": {
        "label": "Skills:",
        "type": "checkbox",
        "options": ["Python", "Machine Learning", "Web Development"]
    },
    "feedback": {
        "label": "Feedback:",
        "type": "textarea"
    },
    "signature": {
        "label": "Signature:",
        "type": "signature"
    }
}


In [ ]:
def is_same_line(b1, b2, tolerance=10):
    return abs(b1[1] - b2[1]) < tolerance

def is_right_of(label_box, word_box):
    return word_box[0] > label_box[2]

def clean(text):
    return text.replace("()", "").strip()


In [ ]:
def extract_key_values(ocr_words, schema):
    result = {}

    for field, info in schema.items():
        label = info["label"]
        field_type = info["type"]

        label_box = None
        for word, box in ocr_words:
            if label in word:
                label_box = box
                break

        if not label_box:
            result[field] = None
            continue

        # TEXT FIELD
        if field_type == "text":
            for word, box in ocr_words:
                if is_same_line(label_box, box) and is_right_of(label_box, box):
                    result[field] = clean(word)
                    break

        # RADIO / CHECKBOX
        elif field_type in ["radio", "checkbox"]:
            selected = []
            for word, box in ocr_words:
                if any(opt in word for opt in info["options"]):
                    selected.append(clean(word))
            result[field] = selected if field_type == "checkbox" else selected[0] if selected else None

        # TEXTAREA
        elif field_type == "textarea":
            for word, box in ocr_words:
                if is_right_of(label_box, box):
                    result[field] = clean(word)
                    break

        # SIGNATURE
        elif field_type == "signature":
            result[field] = "PRESENT"

    return result


In [ ]:
ocr_words = [
    ('Al', [120, 89, 136, 98]),
    ('Accessible', [142, 89, 235, 98]),
    ('PDF', [242, 89, 276, 98]),
    ('Form', [282, 89, 324, 98]),

    ('Name:', [122, 178, 177, 187]),

    ('Gender:', [122, 237, 189, 247]),
    ('()Male', [122, 267, 181, 279]),
    ('()', [200, 267, 214, 279]),
    ('Female', [217, 267, 282, 279]),
    ('()', [286, 265, 314, 280]),
    ('Other', [321, 267, 370, 279]),

    ('Skills:', [121, 326, 171, 336]),
    ('[]', [122, 356, 135, 368]),
    ('Python', [142, 356, 200, 368]),
    ('[]', [122, 386, 135, 398]),
    ('Machine', [142, 386, 214, 395]),
    ('Learning', [221, 386, 294, 398]),
    ('[]', [122, 415, 135, 427]),
    ('Web', [141, 415, 179, 424]),
    ('Development', [186, 415, 301, 427]),

    ('Feedback:', [122, 474, 210, 484]),
    (' ', [121, 513, 463, 516]),

    ('Signature:', [121, 563, 207, 575])
]


In [ ]:
print("ocr_words exists:", 'ocr_words' in globals())
print("FIELD_SCHEMA exists:", 'FIELD_SCHEMA' in globals())
print("extract_key_values exists:", 'extract_key_values' in globals())


ocr_words exists: True
FIELD_SCHEMA exists: True
extract_key_values exists: True


In [ ]:
extracted_data = extract_key_values(ocr_words, FIELD_SCHEMA)
extracted_data


{'gender': 'Male',
 'skills': ['Python'],
 'feedback': 'PDF',
 'signature': 'PRESENT'}

In [ ]:
label_list = [
    "O",                # outside
    "FIELD_NAME",       # Name:, Gender:, Skills:
    "TEXT_VALUE",       # John, Python, etc.
    "CHECKBOX",         # [] () ☐ ☑
]

id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in enumerate(label_list)}


In [ ]:
import torch

logits = outputs.logits                 # [1, seq_len, num_labels]
probs = torch.softmax(logits, dim=-1)
predicted_label_ids = torch.argmax(probs, dim=-1)

predicted_labels = [
    id2label[i.item()]
    for i in predicted_label_ids[0]
]

predicted_labels[:30]


['FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'CHECKBOX',
 'CHECKBOX',
 'CHECKBOX',
 'CHECKBOX',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME',
 'FIELD_NAME']

In [ ]:
fused_tokens = []

for (word, box), label in zip(ocr_words, predicted_labels):
    fused_tokens.append({
        "text": word,
        "bbox": box,
        "label": label
    })

fused_tokens[:10]


[{'text': 'Al', 'bbox': [120, 89, 136, 98], 'label': 'FIELD_NAME'},
 {'text': 'Accessible', 'bbox': [142, 89, 235, 98], 'label': 'FIELD_NAME'},
 {'text': 'PDF', 'bbox': [242, 89, 276, 98], 'label': 'FIELD_NAME'},
 {'text': 'Form', 'bbox': [282, 89, 324, 98], 'label': 'FIELD_NAME'},
 {'text': 'Name:', 'bbox': [122, 178, 177, 187], 'label': 'FIELD_NAME'},
 {'text': 'Gender:', 'bbox': [122, 237, 189, 247], 'label': 'FIELD_NAME'},
 {'text': '()Male', 'bbox': [122, 267, 181, 279], 'label': 'FIELD_NAME'},
 {'text': '()', 'bbox': [200, 267, 214, 279], 'label': 'FIELD_NAME'},
 {'text': 'Female', 'bbox': [217, 267, 282, 279], 'label': 'FIELD_NAME'},
 {'text': '()', 'bbox': [286, 265, 314, 280], 'label': 'FIELD_NAME'}]

In [ ]:
def rule_fusion_engine(tokens):
    data = {
        "name": None,
        "gender": None,
        "skills": [],
        "feedback": None,
        "signature": None
    }

    current_field = None

    for t in tokens:
        text = t["text"].strip()
        label = t["label"]

        # Detect field headers
        if text.lower().startswith("name"):
            current_field = "name"
        elif text.lower().startswith("gender"):
            current_field = "gender"
        elif text.lower().startswith("skills"):
            current_field = "skills"
        elif text.lower().startswith("feedback"):
            current_field = "feedback"
        elif text.lower().startswith("signature"):
            current_field = "signature"

        # Detect checkboxes
        if "()" in text or "[]" in text:
            continue  # box itself, skip

        # Assign values
        if current_field == "gender" and text.lower() in ["male", "female", "other"]:
            data["gender"] = text

        if current_field == "skills" and text not in ["Skills:", "[]"]:
            if len(text) > 2:
                data["skills"].append(text)

    if data["signature"] is None:
        data["signature"] = "PRESENT"

    return data


In [ ]:
final_output = rule_fusion_engine(fused_tokens)
final_output


{'name': None,
 'gender': 'Other',
 'skills': ['Python', 'Machine', 'Learning', 'Web', 'Development'],
 'feedback': None,
 'signature': 'PRESENT'}

In [ ]:
def normalize_tokens(fused_tokens):
    clean = []
    for t in fused_tokens:
        text = t["text"].strip()
        if text == "":
            continue
        clean.append({
            "text": text,
            "bbox": t["bbox"],
            "label": t["label"]
        })
    return clean

tokens = normalize_tokens(fused_tokens)
tokens[:10]


[{'text': 'Al', 'bbox': [120, 89, 136, 98], 'label': 'FIELD_NAME'},
 {'text': 'Accessible', 'bbox': [142, 89, 235, 98], 'label': 'FIELD_NAME'},
 {'text': 'PDF', 'bbox': [242, 89, 276, 98], 'label': 'FIELD_NAME'},
 {'text': 'Form', 'bbox': [282, 89, 324, 98], 'label': 'FIELD_NAME'},
 {'text': 'Name:', 'bbox': [122, 178, 177, 187], 'label': 'FIELD_NAME'},
 {'text': 'Gender:', 'bbox': [122, 237, 189, 247], 'label': 'FIELD_NAME'},
 {'text': '()Male', 'bbox': [122, 267, 181, 279], 'label': 'FIELD_NAME'},
 {'text': '()', 'bbox': [200, 267, 214, 279], 'label': 'FIELD_NAME'},
 {'text': 'Female', 'bbox': [217, 267, 282, 279], 'label': 'FIELD_NAME'},
 {'text': '()', 'bbox': [286, 265, 314, 280], 'label': 'FIELD_NAME'}]

In [ ]:
def is_checked(token_text):
    token_text = token_text.strip()
    if token_text.startswith(("()", "[]")) and len(token_text) > 2:
        return True
    return False


In [ ]:
def merge_words(tokens, x_gap=20):
    merged = []
    buffer = None

    for t in tokens:
        if buffer is None:
            buffer = t
            continue

        same_line = abs(buffer["bbox"][1] - t["bbox"][1]) < 10
        close_x = t["bbox"][0] - buffer["bbox"][2] < x_gap

        if same_line and close_x:
            buffer["text"] += " " + t["text"]
            buffer["bbox"][2] = t["bbox"][2]
        else:
            merged.append(buffer)
            buffer = t

    if buffer:
        merged.append(buffer)

    return merged


In [ ]:
merged_tokens = merge_words(tokens)
merged_tokens[:15]


[{'text': 'Al Accessible PDF Form',
  'bbox': [120, 89, 324, 98],
  'label': 'FIELD_NAME'},
 {'text': 'Name:', 'bbox': [122, 178, 177, 187], 'label': 'FIELD_NAME'},
 {'text': 'Gender:', 'bbox': [122, 237, 189, 247], 'label': 'FIELD_NAME'},
 {'text': '()Male () Female () Other',
  'bbox': [122, 267, 370, 279],
  'label': 'FIELD_NAME'},
 {'text': 'Skills:', 'bbox': [121, 326, 171, 336], 'label': 'CHECKBOX'},
 {'text': '[] Python', 'bbox': [122, 356, 200, 368], 'label': 'CHECKBOX'},
 {'text': '[] Machine Learning',
  'bbox': [122, 386, 294, 398],
  'label': 'CHECKBOX'},
 {'text': '[] Web Development',
  'bbox': [122, 415, 301, 427],
  'label': 'FIELD_NAME'},
 {'text': 'Feedback:', 'bbox': [122, 474, 210, 484], 'label': 'FIELD_NAME'},
 {'text': 'Signature:', 'bbox': [121, 563, 207, 575], 'label': 'FIELD_NAME'}]

In [ ]:
def extract_form_data(tokens):
    data = {
        "name": None,
        "gender": None,
        "skills": [],
        "feedback": None,
        "signature": None
    }

    current_field = None

    for t in tokens:
        text = t["text"].lower()

        # Field headers
        if "name" in text:
            current_field = "name"
            continue
        if "gender" in text:
            current_field = "gender"
            continue
        if "skills" in text:
            current_field = "skills"
            continue
        if "feedback" in text:
            current_field = "feedback"
            continue
        if "signature" in text:
            data["signature"] = "PRESENT"
            continue

        # Gender checkbox
        if current_field == "gender":
            for g in ["male", "female", "other"]:
                if g in text and is_checked(t["text"]):
                    data["gender"] = g.capitalize()

        # Skills checkbox
        if current_field == "skills" and is_checked(t["text"]):
            skill = t["text"].replace("[]", "").strip()
            if len(skill) > 2:
                data["skills"].append(skill)

    return data


In [ ]:
final_tokens = merge_words(normalize_tokens(fused_tokens))
final_output = extract_form_data(final_tokens)
final_output


{'name': None,
 'gender': 'Other',
 'skills': ['Python', 'Machine Learning', 'Web Development'],
 'feedback': None,
 'signature': 'PRESENT'}

In [ ]:
def generate_html_form(form_data):
    html = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Editable Accessible Form</title>
<style>
  body { font-family: Arial; padding: 20px; }
  label { font-weight: bold; }
  .field { margin-bottom: 15px; }
</style>
</head>
<body>

<h2>Accessible PDF Form (Editable)</h2>

<form>
"""
    # Name
    html += f"""
<div class="field">
<label>Name:</label><br>
<input type="text" value="{form_data.get('name') or ''}" />
</div>
"""

    # Gender
    html += "<div class='field'><label>Gender:</label><br>"
    for g in ["Male", "Female", "Other"]:
        checked = "checked" if form_data["gender"] == g else ""
        html += f"""
<input type="radio" name="gender" {checked}> {g}<br>
"""
    html += "</div>"

    # Skills
    html += "<div class='field'><label>Skills:</label><br>"
    for skill in ["Python", "Machine Learning", "Web Development"]:
        checked = "checked" if skill in form_data["skills"] else ""
        html += f"""
<input type="checkbox" {checked}> {skill}<br>
"""
    html += "</div>"

    # Feedback
    html += f"""
<div class="field">
<label>Feedback:</label><br>
<textarea rows="4" cols="50">{form_data.get('feedback') or ''}</textarea>
</div>
"""

    # Signature
    html += """
<div class="field">
<label>Signature:</label><br>
<input type="text" placeholder="Signed" />
</div>
"""

    html += """
<button type="submit">Save</button>
<button type="button" onclick="window.print()">Print</button>

</form>
</body>
</html>
"""
    return html


In [ ]:
html_form = generate_html_form(final_output)

with open("editable_form.html", "w") as f:
    f.write(html_form)

print("✅ Editable form generated: editable_form.html")


✅ Editable form generated: editable_form.html


In [ ]:
from IPython.display import FileLink

FileLink("editable_form.html")


/content/editable_form.html

In [ ]:
!ls


'dummy pdf (1).pdf'  'dummy pdf.pdf'	   page_1.png
'dummy pdf (2).pdf'   editable_form.html   sample_data


In [ ]:
with open("editable_form.html", "w") as f:
    f.write(html_form)


In [ ]:
import webbrowser
webbrowser.open("editable_form.html")


False

In [ ]:
import os, webbrowser

file_path = os.path.abspath("editable_form.html")
webbrowser.open(f"file:///{file_path}")


False

In [ ]:
import os
os.getcwd()


'/content'

In [ ]:
os.listdir()


['.config', 'sample_data']

In [ ]:
webbrowser.open("file:///editable_form.html")


False

In [ ]:
final_output = {
    "name": None,
    "gender": "Male",
    "skills": ["Python", "Machine Learning", "Web Development"],
    "feedback": None,
    "signature": "PRESENT"
}


In [ ]:
def generate_html_form(form_data):
    html = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Editable Accessible Form</title>
<style>
  body { font-family: Arial, sans-serif; padding: 20px; }
  label { font-weight: bold; }
  .field { margin-bottom: 15px; }
</style>
</head>
<body>

<h2>Accessible PDF Form (Editable)</h2>

<form>
"""
    # Name
    html += f"""
<div class="field">
<label>Name:</label><br>
<input type="text" value="{form_data.get('name') or ''}">
</div>
"""

    # Gender
    html += "<div class='field'><label>Gender:</label><br>"
    for g in ["Male", "Female", "Other"]:
        checked = "checked" if form_data.get("gender") == g else ""
        html += f"""
<input type="radio" name="gender" {checked}> {g}<br>
"""
    html += "</div>"

    # Skills
    html += "<div class='field'><label>Skills:</label><br>"
    for skill in ["Python", "Machine Learning", "Web Development"]:
        checked = "checked" if skill in form_data.get("skills", []) else ""
        html += f"""
<input type="checkbox" {checked}> {skill}<br>
"""
    html += "</div>"

    # Feedback
    html += f"""
<div class="field">
<label>Feedback:</label><br>
<textarea rows="4" cols="50">{form_data.get('feedback') or ''}</textarea>
</div>
"""

    # Signature
    html += """
<div class="field">
<label>Signature:</label><br>
<input type="text" placeholder="Signed">
</div>
"""

    html += """
<button type="submit">Save</button>
<button type="button" onclick="window.print()">Print</button>

</form>
<button onclick="window.print()">Print / Save as PDF</button>

</body>
</html>
"""

    return html


In [ ]:
html_form = generate_html_form(final_output)

with open("editable_form.html", "w", encoding="utf-8") as f:
    f.write(html_form)

print("✅ Editable form generated: editable_form.html")


✅ Editable form generated: editable_form.html


In [ ]:
import os, webbrowser

file_path = os.path.abspath("editable_form.html")
print("Opening:", file_path)
webbrowser.open(f"file:///{file_path}")


Opening: /content/editable_form.html


False

In [ ]:
!ls -l /content/editable_form.html


-rw-r--r-- 1 root root 1112 Feb 17 17:39 /content/editable_form.html


In [ ]:
from google.colab import files
files.download('/content/editable_form.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install pdf2image pillow


In [ ]:
from pdf2image import convert_from_path
